In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
f= open('/data/tiny-imagenet-200/original/val/val_annotations.txt')

# array=[]

# for line in f.readlines():
#     array.append([])
#     for i in line.split():
#         array[-1].append(i)

# print len(array)
# # print array

In [3]:
import os, shutil

original_dataset_dir = '/data/tiny-imagenet-200/original/val/images'
train_dir = '/data/tiny-imagenet-200/train'
val_dir = '/data/tiny-imagenet-200/val'

base_dir = '/data/tiny-imagenet-200/val'
if not os.path.exists(base_dir):os.mkdir(base_dir)
    
#train,validation,test dir
# validation_dir=os.path.join(base_dir,'images')
# if not os.path.exists(validation_dir): os.mkdir(validation_dir)
    
# for i in range(10000):
#     dir = os.path.join(base_dir,array[i][1])
    
#     fname = array[i][0]
# #     print array[i][0],array[i][1]
#     if not os.path.exists(dir):os.mkdir(dir)
#     src=os.path.join(original_dataset_dir,fname)
#     dst=os.path.join(dir,fname)
#     shutil.copyfile(src,dst)
    

    
    


In [4]:
#pre processing

train_dir = '/data/tiny-imagenet-200/train'
val_dir = '/data/tiny-imagenet-200/val'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10,width_shift_range =0.1,height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(64,64),batch_size=200,class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(val_dir,target_size=(64,64),batch_size=200,class_mode='categorical')



Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [5]:
# model
from keras import layers
from keras import models
from keras import regularizers
from keras.layers.normalization import BatchNormalization


model=models.Sequential()

model.add(layers.convolutional.ZeroPadding2D((1,1),input_shape=(64,64,3)))
# model.add(layers.Conv2D(64,(3,3),activation='relu'))
# model.add(BatchNormalization())
# model.add(layers.Conv2D(64,(3,3),activation='relu'))
# model.add(BatchNormalization())
# model.add(layers.MaxPooling2D((2,2)))

# model.add(layers.convolutional.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.convolutional.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.convolutional.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(512,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(512,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(512,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(512,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((6,6)))

model.add(layers.Flatten())
model.add(layers.Dense(1024,kernel_regularizer=regularizers.l2(0.001),activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dense(200,kernel_regularizer=regularizers.l2(0.001),activation='softmax'))
# model.add(BatchNormalization())


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 66, 66, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 62, 62, 128)       512       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 128)       147584    
_________________________________________________________________
batch_normalization_3 (Batch (None, 60, 60, 128)       512       
__________

In [6]:
## training and save

from keras import optimizers
from keras.callbacks import ModelCheckpoint
import os, shutil

# learning rate log
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

optimizer = optimizers.RMSprop(lr=1e-3)
lr_metric = get_lr_metric(optimizer)

model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc',lr_metric])

# check point
log_dir='/data/tiny_log'
log_data_dir=os.path.join(log_dir,'log_5_3') ############################################# version
if not os.path.exists(log_data_dir): os.mkdir(log_data_dir)

filepath=os.path.join(log_data_dir,'weights.{epoch:02d}-{val_acc:.2f}.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

callbacks_list = [checkpoint]

#training
history=model.fit_generator(train_generator,epochs=50,steps_per_epoch=500,callbacks=callbacks_list, validation_data=validation_generator,validation_steps=50)

#model save
model_weight=os.path.join(log_data_dir,'tiny_imagenet.h5')
model_arch=os.path.join(log_data_dir,'tiny_imagenet.json')

model.save(model_weight)
with open(model_arch,'w') as f:
    f.write(model.to_json())

import csv

dict = history.history

csv_path=os.path.join(log_data_dir,'tiny_log.csv')

w = csv.writer(open(csv_path, "w"))
for key, val in dict.items():
    w.writerow([key, val])

Epoch 1/50
500/500 [==============================] - 467s 934ms/step - loss: 5.5032 - acc: 0.0438 - lr: 0.0010 - val_loss: 5.1487 - val_acc: 0.0645 - val_lr: 0.0010
Epoch 2/50
500/500 [==============================] - 489s 978ms/step - loss: 4.4506 - acc: 0.1160 - lr: 0.0010 - val_loss: 4.9067 - val_acc: 0.0751 - val_lr: 0.0010
Epoch 3/50
500/500 [==============================] - 490s 980ms/step - loss: 4.0533 - acc: 0.1729 - lr: 0.0010 - val_loss: 4.5406 - val_acc: 0.1109 - val_lr: 0.0010
Epoch 4/50
500/500 [==============================] - 490s 980ms/step - loss: 3.8181 - acc: 0.2154 - lr: 0.0010 - val_loss: 4.0249 - val_acc: 0.1909 - val_lr: 0.0010
Epoch 5/50
500/500 [==============================] - 490s 980ms/step - loss: 3.6646 - acc: 0.2503 - lr: 0.0010 - val_loss: 4.0758 - val_acc: 0.1873 - val_lr: 0.0010
Epoch 6/50
500/500 [==============================] - 491s 982ms/step - loss: 3.5349 - acc: 0.2792 - lr: 0.0010 - val_loss: 4.2039 - val_acc: 0.1912 - val_lr: 0.0010
Epoc

In [7]:
import csv

dict = history.history

csv_path=os.path.join(log_data_dir,'tiny_log.csv')

w = csv.writer(open(csv_path, "w"))
for key, val in dict.items():
    w.writerow([key, val])

In [8]:
# for layer in model.layers:
#     print (layer, '\n')
# #     print(layer., layer.get_weights())


In [9]:
import matplotlib.pyplot as plt

# acc = history.history['acc']
# val_acc = history.history['val_acc']

# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(acc)+1)

# plt.plot(epochs, acc,'bo',label = 'Training accuracy')
# plt.plot(epochs, val_acc,'b',label = 'Validation accuracy')
# plt.title('Training and Validation accuracy')
# plt.legend()

# plt.figure()

# plt.plot(epochs,loss,'bo',label = 'Training loss')
# plt.plot(epochs,val_loss,'b',label = 'Validation loss')
# plt.title('Training and Validation loss')
# plt.legend()

# plt.show()

# # with open('/home/dkkim/documents/tiny_log.txt','w') as output:
# #     output.write(str(history.history))

# model save
model_weight=os.path.join(log_data_dir,'tiny_imagenet.h5')
model_arch=os.path.join(log_data_dir,'tiny_imagenet.json')





In [10]:
# from keras.preprocessing import image
# import matplotlib.pyplot as plt

# path_check='/home/dkkim/downloads/tiny-imagenet-200/train/n01644900/images'
# fnames = sorted([os.path.join(path_check, fname) for fname in os.listdir(path_check)])


# img_path = fnames[2]

# img = image.load_img(img_path, target_size=(64,64))

# x = image.img_to_array(img)
# x = x.reshape((1,) + x.shape)

# datagen = ImageDataGenerator(rotation_range=10,width_shift_range =0.1,height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,fill_mode='nearest')

# j=0
# for batch in datagen.flow(x, batch_size =1):
#     plt.figure(j)
#     imgplot = plt.imshow(image.array_to_img(batch[0]))
#     j += 1
#     if j%4 ==0:
#         break
        
# plt.show

In [11]:
# #visualization
# img_path = '/home/dkkim/downloads/tiny-imagenet-200/original/train/n01443537/images/n01443537_1.JPEG'

# from keras.preprocessing import image
# import numpy as np

# img = image.load_img(img_path, target_size=(64,64))
# img_tensor =image.img_to_array(img)
# img_tensor=np.expand_dims(img_tensor,axis=0)
# img_tensor /= 255

# print(img_tensor.shape)

# import matplotlib.pyplot as plt

# # original image show
# plt.imshow(img_tensor[0])
# plt.show()

# layer_outputs = [layer.output for layer in model.layers[10:]]
# activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

# activations = activation_model.predict(img_tensor)

# layer_names = []
# for layer in model.layers[:8]:
#     layer_names.append(layer.name)
    
# images_per_row = 16

# for layer_name, layer_activation in zip(layer_names,activations):
#     n_features = layer_activation.shape[-1]
    
#     size = layer_activation.shape[1]
    
#     n_cols = n_features // images_per_row
#     display_grid = np.zeros((size*n_cols,images_per_row*size))
#     layer_activation -= layer_activation.mean()
#     layer_activation /= layer_activation.std()
# #     layer_activation *= 64
# #     layer_activation += 128
# #     layer_activation = np.clip(layer_activation, 0, 255).astype('uint8')

#     for col in range(n_cols):
#         for row in range(images_per_row):
#             channel_image = layer_activation[0,:,:,col*images_per_row+row]
# #             channel_image -= channel_image.mean()
# #             channel_image /= channel_image.std()
# #             channel_image *= 64
# #             channel_image += 128
#             channel_image = np.clip(channel_image,0,255).astype('uint8')
#             display_grid[col * size : (col + 1) * size, row * size : (row + 1)*size] = channel_image
    
#     scale = 1. / size
#     plt.figure(figsize=(scale * display_grid.shape[1],scale * display_grid.shape[0]))
#     plt.title(layer_name)
#     plt.grid(False)
#     plt.imshow(display_grid,aspect='auto',cmap='viridis',clim=(layer_activation.min(),layer_activation.max())) # clim = (min, max)
    
# plt.show()


In [12]:
# filter visualization



In [13]:
from keras import backend as K 

# Do some code, e.g. train and save model

K.clear_session()

del model